In [1]:
import nest_asyncio
nest_asyncio.apply()
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

In [2]:

ollama_model = OpenAIModel(
    model_name='qwen2.5-coder:14b', provider=OpenAIProvider(base_url='http://localhost:11434/v1')
)

roulette_agent = Agent(  
    ollama_model,
    deps_type=int,
    result_type=bool,
    system_prompt=(
        'Use the `roulette_wheel` function to see if the '
        'customer has won based on the number they provide.'
    )
)

In [3]:
ROULETTE_NUMBERS = 37
import random

@roulette_agent.tool
def roulette_wheel(ctx: RunContext[int], number: int) -> bool:
    # generate random numbers between 0 and 36
    winning_numbers = [random.randint(0, ROULETTE_NUMBERS) for _ in range(19)]

    # if 0 < number < ROULETTE_NUMBERS:
    #     raise ValueError(f'Number must be between 0 and {ROULETTE_NUMBERS}')
    print(f'Winning numbers: {winning_numbers}')
    print(f'User number: {number}')
    print(f'deps: {ctx.deps}')
    return number in winning_numbers

In [4]:
result = roulette_agent.run_sync("is fifteen a winning number", deps=15)

Winning numbers: [1, 34, 17, 32, 1, 1, 6, 22, 11, 36, 24, 1, 36, 6, 34, 0, 20, 5, 31]
User number: 15
deps: 15


In [5]:
result

AgentRunResult(data=False)

using iter

In [6]:
import nest_asyncio
nest_asyncio.apply()
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
import pprint as pp

In [7]:
from pydantic_ai import Agent

ollama_model = OpenAIModel(
    model_name='qwen2.5-coder:14b', provider=OpenAIProvider(base_url='http://localhost:11434/v1')
)
agent = Agent(ollama_model)


nodes = []

# Begin an AgentRun, which is an async-iterable over the nodes of the agent's graph
async with agent.iter('What is the capital of France?') as agent_run:
    async for node in agent_run:
        # Each node represents a step in the agent's execution
        nodes.append(node)
pp.pprint(nodes)
"""
    [
        ModelRequestNode(
            request=ModelRequest(
                parts=[
                    UserPromptPart(
                        content='What is the capital of France?',
                        timestamp=datetime.datetime(...),
                        part_kind='user-prompt',
                    )
                ],
                kind='request',
            )
        ),
        CallToolsNode(
            model_response=ModelResponse(
                parts=[TextPart(content='Paris', part_kind='text')],
                model_name='gpt-4o',
                timestamp=datetime.datetime(...),
                kind='response',
            )
        ),
        End(data=FinalResult(data='Paris', tool_name=None, tool_call_id=None)),
    ]
"""

pp.pprint(agent_run.result.data)
#> Paris

[ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='What is '
                                                                     'the '
                                                                     'capital '
                                                                     'of '
                                                                     'France?',
                                                             timestamp=datetime.datetime(2025, 3, 7, 19, 25, 13, 25883, tzinfo=datetime.timezone.utc),
                                                             part_kind='user-prompt')],
                                       kind='request')),
 CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='The '
                                                                    'capital '
                                                                    'of France '
                                                                    'is Paris.

In [8]:
import nest_asyncio
nest_asyncio.apply()
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
import pprint as pp

In [9]:
from dataclasses import dataclass

import httpx

from pydantic_ai import Agent
ollama_model = OpenAIModel(
    model_name='qwen2.5-coder:14b', provider=OpenAIProvider(base_url='http://localhost:11434/v1')
)
agent = Agent(ollama_model)

@dataclass
class MyDeps:  
    api_key: str
    http_client: httpx.AsyncClient



async with httpx.AsyncClient() as client:
    deps = MyDeps('foobar', client)
    result = await agent.run(
        'Tell me a joke.',
        deps=deps,  
    )
    pp.pprint(result.data)
    #> Did you hear about the toothpaste scandal? They called it Colgate.

("Sure, here's a light-hearted joke for you:\n"
 '\n'
 'Why did the tomato turn red?\n'
 '\n'
 'Because it saw the salad dressing! \n'
 '\n'
 "I hope you found that amusing! If you'd like to hear more jokes or have any "
 'other questions, feel free to ask.')


In [10]:
@dataclass
class MyDeps:
    api_key: str
    http_client: httpx.AsyncClient


agent = Agent(
    ollama_model,
    deps_type=MyDeps,
)


@agent.system_prompt  
async def get_system_prompt(ctx: RunContext[MyDeps]) -> str:  
    response = await ctx.deps.http_client.get(  
        'https://example.com',
        headers={'Authorization': f'Bearer {ctx.deps.api_key}'},  
    )
    response.raise_for_status()
    return f'Prompt: {response.text}'


async with httpx.AsyncClient() as client:
    deps = MyDeps('foobar', client)
    result = await agent.run('Tell me a joke.', deps=deps)
    pp.pprint(result.data)
        #> Did you hear about the toothpaste scandal? They called it Colgate.

"Why couldn't the bicycle stand up by itself? Because it was two-tired!"


In [11]:
import nest_asyncio
nest_asyncio.apply()
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
import pprint as pp

In [12]:
from pydantic_ai.common_tools.duckduckgo import duckduckgo_search_tool
from pydantic_ai import Agent
ollama_model = OpenAIModel(
    model_name='qwq:32b', provider=OpenAIProvider(base_url='http://localhost:11434/v1')
)
agent = Agent(ollama_model)
agent = Agent(
    ollama_model,
    tools=[duckduckgo_search_tool()],
    system_prompt='Search DuckDuckGo for the given query and return the results.',
)

result = agent.run_sync(
    'Can you list the top five highest-grossing animated films of 2025?'
)
print(result.data)
"""
I looked into several sources on animated box‐office performance in 2025, and while detailed
rankings can shift as more money is tallied, multiple independent reports have already
highlighted a couple of record‐breaking shows. For example:

• Ne Zha 2 – News outlets (Variety, Wikipedia's "List of animated feature films of 2025", and others)
    have reported that this Chinese title not only became the highest‑grossing animated film of 2025
    but also broke records as the highest‑grossing non‑English animated film ever. One article noted
    its run exceeded US$1.7 billion.
• Inside Out 2 – According to data shared on Statista and in industry news, this Pixar sequel has been
    on pace to set new records (with some sources even noting it as the highest‑grossing animated film
    ever, as of January 2025).

Beyond those two, some entertainment trade sites (for example, a Just Jared article titled
"Top 10 Highest-Earning Animated Films at the Box Office Revealed") have begun listing a broader
top‑10. Although full consolidated figures can sometimes differ by source and are updated daily during
a box‑office run, many of the industry trackers have begun to single out five films as the biggest
earners so far in 2025.

Unfortunately, although multiple articles discuss the "top animated films" of 2025, there isn't yet a
single, universally accepted list with final numbers that names the complete top five. (Box‑office
rankings, especially mid‑year, can be fluid as films continue to add to their totals.)

Based on what several sources note so far, the two undisputed leaders are:
1. Ne Zha 2
2. Inside Out 2

The remaining top spots (3–5) are reported by some outlets in their "Top‑10 Animated Films"
lists for 2025 but the titles and order can vary depending on the source and the exact cut‑off
date of the data. For the most up‑to‑date and detailed ranking (including the 3rd, 4th, and 5th
highest‑grossing films), I recommend checking resources like:
• Wikipedia's "List of animated feature films of 2025" page
• Box‑office tracking sites (such as Box Office Mojo or The Numbers)
• Trade articles like the one on Just Jared

To summarize with what is clear from the current reporting:
1. Ne Zha 2
2. Inside Out 2
3–5. Other animated films (yet to be definitively finalized across all reporting outlets)

If you're looking for a final, consensus list of the top five, it may be best to wait until
the 2025 year‑end box‑office tallies are in or to consult a regularly updated entertainment industry source.

Would you like help finding a current source or additional details on where to look for the complete updated list?
"""

<think>
Alright, let me tackle this query about the top five highest-grossing animated films of 2025. The user is asking for a list that's specific to 2025, which means I need to ensure the data provided pertains accurately to that year.

First, looking at the tool responses from DuckDuckGo, several sources are cited. Wikipedia mentions "Ne Zha 2" surpassing other films and becoming the highest-grossing animated film as of early 2025. Another entry on JustJared lists "Ne Zha 2" with a gross of $1.72 billion as of February 20, 2025. This suggests it's at the top.

Next, there are mentions of other films like "Inside Out 2" being noted in one source as the highest even though Ne Zha 2 overtook it. The Wikipedia entry about box office number ones in the U.S. lists "Captain America: Brave New World" as high-grossing domestically for Disney, but since that's a live-action film (part of Captain America franchise), unless it's an animated/live action hybrid, it might not count towards the pur

'\nI looked into several sources on animated box‐office performance in 2025, and while detailed\nrankings can shift as more money is tallied, multiple independent reports have already\nhighlighted a couple of record‐breaking shows. For example:\n\n• Ne Zha 2 – News outlets (Variety, Wikipedia\'s "List of animated feature films of 2025", and others)\n    have reported that this Chinese title not only became the highest‑grossing animated film of 2025\n    but also broke records as the highest‑grossing non‑English animated film ever. One article noted\n    its run exceeded US$1.7 billion.\n• Inside Out 2 – According to data shared on Statista and in industry news, this Pixar sequel has been\n    on pace to set new records (with some sources even noting it as the highest‑grossing animated film\n    ever, as of January 2025).\n\nBeyond those two, some entertainment trade sites (for example, a Just Jared article titled\n"Top 10 Highest-Earning Animated Films at the Box Office Revealed") have

In [15]:
! uv pip install 'pydantic-ai-slim[tavily]'
! uv pip install 'pydantic-ai'

Using Python 3.12.3 environment at: /home/yi/Documents/MCP/fastmcp/.venv
Audited 1 package in 8ms


Using Python 3.12.3 environment at: /home/yi/Documents/MCP/fastmcp/.venv
Audited 1 package in 8ms


In [16]:
import os

from pydantic_ai.agent import Agent
from pydantic_ai.common_tools.tavily import tavily_search_tool

api_key=os.environ['TAVILY_API_KEY']



agent = Agent(
    ollama_model,
    tools=[tavily_search_tool(api_key)],
    system_prompt='Search Tavily for the given query and return the results.',
)

result = agent.run_sync('Tell me the top news in the GenAI world, give me links.')
print(result.data)
"""
Here are some of the top recent news articles related to GenAI:

1. How CLEAR users can improve risk analysis with GenAI – Thomson Reuters
   Read more: https://legal.thomsonreuters.com/blog/how-clear-users-can-improve-risk-analysis-with-genai/
   (This article discusses how CLEAR's new GenAI-powered tool streamlines risk analysis by quickly summarizing key information from various public data sources.)

2. TELUS Digital Survey Reveals Enterprise Employees Are Entering Sensitive Data Into AI Assistants More Than You Think – FT.com
   Read more: https://markets.ft.com/data/announce/detail?dockey=600-202502260645BIZWIRE_USPRX____20250226_BW490609-1
   (This news piece highlights findings from a TELUS Digital survey showing that many enterprise employees use public GenAI tools and sometimes even enter sensitive data.)

3. The Essential Guide to Generative AI – Virtualization Review
   Read more: https://virtualizationreview.com/Whitepapers/2025/02/SNOWFLAKE-The-Essential-Guide-to-Generative-AI.aspx
   (This guide provides insights into how GenAI is revolutionizing enterprise strategies and productivity, with input from industry leaders.)

Feel free to click on the links to dive deeper into each story!
"""

UnexpectedModelBehavior: Tool exceeded max retries count of 1

In [17]:
result.usage()

Usage(requests=2, request_tokens=1655, response_tokens=1901, total_tokens=3556, details=None)

In [18]:
pp.pprint(result.data)

('<think>\n'
 'Alright, let me tackle this query about the top five highest-grossing '
 "animated films of 2025. The user is asking for a list that's specific to "
 '2025, which means I need to ensure the data provided pertains accurately to '
 'that year.\n'
 '\n'
 'First, looking at the tool responses from DuckDuckGo, several sources are '
 'cited. Wikipedia mentions "Ne Zha 2" surpassing other films and becoming the '
 'highest-grossing animated film as of early 2025. Another entry on JustJared '
 'lists "Ne Zha 2" with a gross of $1.72 billion as of February 20, 2025. This '
 "suggests it's at the top.\n"
 '\n'
 'Next, there are mentions of other films like "Inside Out 2" being noted in '
 'one source as the highest even though Ne Zha 2 overtook it. The Wikipedia '
 'entry about box office number ones in the U.S. lists "Captain America: Brave '
 'New World" as high-grossing domestically for Disney, but since that\'s a '
 "live-action film (part of Captain America franchise), unless

Message history

In [19]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='Search DuckDuckGo for the given query and return the results.', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='Can you list the top five highest-grossing animated films of 2025?', timestamp=datetime.datetime(2025, 3, 7, 19, 25, 23, 42614, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[TextPart(content='', part_kind='text'), ToolCallPart(tool_name='duckduckgo_search', args='{"query":"top five highest-grossing animated films of 2025"}', tool_call_id='call_fzfnjuvc', part_kind='tool-call')], model_name='qwq:32b', timestamp=datetime.datetime(2025, 3, 7, 19, 26, 8, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='duckduckgo_search', content=[{'title': 'List of animated feature films of 2025 - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2025', 'body': "Ne Zha 2 surpassed The Battle at La

## Agent delegation
"Agent delegation" refers to the scenario where an agent delegates work to another agent, then takes back control when the delegate agent (the agent called from within a tool) finishes.

Since agents are stateless and designed to be global, you do not need to include the agent itself in agent dependencies.

You'll generally want to pass ctx.usage to the usage keyword argument of the delegate agent run so usage within that run counts towards the total usage of the parent agent run.

In [23]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.usage import UsageLimits
from pydantic_ai.models.openai import OpenAIModel
import nest_asyncio
nest_asyncio.apply()

qwen_model = OpenAIModel(
    model_name='qwen2.5-coder:14b', provider=OpenAIProvider(base_url='http://localhost:11434/v1')
)

joke_selection_agent = Agent(  
    qwen_model,
    system_prompt=(
        'Use the `joke_factory` to generate some jokes, then choose the best. '
        'You must return just a single joke.'
    ),
)
joke_generation_agent = Agent(  
    qwen_model, result_type=list[str]
)

@joke_selection_agent.tool
async def joke_factory(ctx: RunContext[None], count: int) -> list[str]:
    r = await joke_generation_agent.run(  
        f'Please generate {count} jokes.'
        # usage=ctx.usage,  
    )
    return r.data  


result = joke_selection_agent.run_sync(
    'Tell me a joke.'
    # usage_limits=UsageLimits(request_limit=5, total_tokens_limit=3000),
)
print(result.data)
#> Did you hear about the toothpaste scandal? They called it Colgate.
print(result.usage())
"""
Usage(
    requests=3, request_tokens=204, response_tokens=24, total_tokens=228, details=None
)
"""

Here's the joke I've generated for you:

Why don't scientists trust atoms? Because they make up everything!

I hope this joke brings a smile to your face! Let me know if you need another one or any other help.
Usage(requests=1, request_tokens=161, response_tokens=48, total_tokens=209, details=None)


'\nUsage(\n    requests=3, request_tokens=204, response_tokens=24, total_tokens=228, details=None\n)\n'

In [24]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='Use the `joke_factory` to generate some jokes, then choose the best. You must return just a single joke.', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='Tell me a joke.', timestamp=datetime.datetime(2025, 3, 7, 19, 38, 8, 156676, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[TextPart(content="Here's the joke I've generated for you:\n\nWhy don't scientists trust atoms? Because they make up everything!\n\nI hope this joke brings a smile to your face! Let me know if you need another one or any other help.", part_kind='text')], model_name='qwen2.5-coder:14b', timestamp=datetime.datetime(2025, 3, 7, 19, 38, 9, tzinfo=datetime.timezone.utc), kind='response')]

Generally the delegate agent needs to either have the same dependencies as the calling agent, or dependencies which are a subset of the calling agent's dependencies.

In [28]:
from dataclasses import dataclass

import httpx

from pydantic_ai import Agent, RunContext
import nest_asyncio
nest_asyncio.apply()

@dataclass
class ClientAndKey:  
    http_client: httpx.AsyncClient
    api_key: str


joke_selection_agent = Agent(
    qwen_model,
    deps_type=ClientAndKey,  
    system_prompt=(
        'Use the `joke_factory` tool to generate some jokes on the given subject, '
        'then choose the best. You must return just a single joke.'
    ),
)
joke_generation_agent = Agent(
    qwen_model,
    deps_type=ClientAndKey,  
    result_type=list[str],
    system_prompt=(
        'Use the "get_jokes" tool to get some jokes on the given subject, '
        'then extract each joke into a list.'
    ),
)


@joke_selection_agent.tool
async def joke_factory(ctx: RunContext[ClientAndKey], count: int) -> list[str]:
    r = await joke_generation_agent.run(
        f'Please generate {count} jokes.',
        deps=ctx.deps
        # usage=ctx.usage,
    )
    return r.data


@joke_generation_agent.tool  
async def get_jokes(ctx: RunContext[ClientAndKey], count: int) -> str:
    response = await ctx.deps.http_client.get(
        'https://example.com',
        params={'count': count},
        headers={'Authorization': f'Bearer {ctx.deps.api_key}'},
    )
    response.raise_for_status()
    return response.text


async with httpx.AsyncClient() as client:
    deps = ClientAndKey(client, 'foobar')
    result = joke_selection_agent.run_sync(
        'Tell me a joke.', 
        deps=deps,
        usage_limits=UsageLimits(total_tokens_limit=3000))
    pp.pprint(result.data)
    #> Did you hear about the toothpaste scandal? They called it Colgate.
    pp.pprint(result.usage())  
    """
    Usage(
        requests=4,
        request_tokens=309,
        response_tokens=32,
        total_tokens=341,
        details=None,
    )
    """

('Here is a joke for you:\n'
 '\n'
 "Why don't scientists trust atoms?\n"
 '\n'
 'Because they make up everything!')
Usage(requests=2,
      request_tokens=360,
      response_tokens=103,
      total_tokens=463,
      details=None)


In [29]:
from typing import Literal, Union
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext
from pydantic_ai.usage import UsageLimits
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.usage import Usage, UsageLimits
from pydantic_ai.messages import ModelMessage
from rich.prompt import Prompt

import nest_asyncio
nest_asyncio.apply()

qwen_model = OpenAIModel(
    model_name='qwen2.5-coder:14b', provider=OpenAIProvider(base_url='http://localhost:11434/v1')
)

In [30]:
class FlightDetails(BaseModel):
    flight_number: str


class Failed(BaseModel):
    """Unable to find a satisfactory choice."""

flight_search_agent = Agent[None, Union[FlightDetails, Failed]](
    qwen_model,
    result_type=Union[FlightDetails, Failed], # type: ingore
    system_prompt=(
        'Use the "flight_search" tool to find a flight '
        'from the given origin to the given destinaiton.'
    )
)

@flight_search_agent.tool
async def flight_search(
    ctx: RunContext[None], origin: str, destinaiton: str
) -> Union[FlightDetails, None]:
    # in reality, this would call a flight search API or
    # use a browser to scrap a flight search website
    return FlightDetails(flight_number='AK46')

usage_limits = UsageLimits(request_limit=60)

async def find_flight(usage: Usage) -> Union[FlightDetails, None]:
    message_history: Union[list[ModelMessage], None] = None
    for _ in range(3):
        prompt = Prompt.ask(
            'Where would you like to fly from and to?'
        )
        result = await flight_search_agent.run(
            prompt,
            message_history=message_history,
            usage=usage,
            usage_limits=usage_limits
        )
        if isinstance(result.data, FlightDetails):
            return result.data
        else:
            message_history = result.all_messages(
                result_tool_return_content='Please try again.'
            )


class SeatPreference(BaseModel):
    row: int = Field(ge=1, le=30)
    seat: Literal['A', 'B', 'C', 'D', 'E', 'F']


# This agent is responsible for extracting the user's seat selection
seat_preference_agent = Agent[None, Union[SeatPreference, Failed]](
    qwen_model,
    result_type=Union[SeatPreference, Failed], # type: ignore
    system_prompt=(
        "Extract the user's seat preference. "
        'Seats A and F are window seats. '
        'Row 1 is the front row and has extra log room. '
        'Rows 14, and 20 also have extra leg roow. '
    )
)

async def find_seat(usage: Usage) -> SeatPreference:
    message_history: Union[list[ModelMessage], None] = None
    while True:
        answer = Prompt.ask('What seat would you like?')

        result = await seat_preference_agent.run(
            answer,
            message_history=message_history,
            usage=usage,
            usage_limits=usage_limits
        )
        if isinstance(result.data, SeatPreference):
            return result.data
        else:
            print('Could not understand seat prefeerence. Please try again.')
            message_history = result.all_messages()

usage: Usage = Usage()

opt_flight_details = await find_flight(usage)

if opt_flight_details is not None:
    print(f'Flight found: {opt_flight_details.flight_number}')
    #> Flight found: AK456
    seat_preference = await find_seat(usage)
    print(f'Seat preference: {seat_preference}')
    #> Seat prerence: row=1 seat='A'

Where would you like to fly from and to?:

Flight found: AK46


What seat would you like?:

Could not understand seat prefeerence. Please try again.


What seat would you like?:

Could not understand seat prefeerence. Please try again.


What seat would you like?:

Could not understand seat prefeerence. Please try again.


What seat would you like?:

Seat preference: row=1 seat='A'


## Programmic agent hand-off
"Programmatic agent hand-off" refers to the scenario where multiple agents are called in succession, with application code and/or a human in the loop responsible for deciding which agent is caleld next.

Here agents don't need to use the same deps.

Here we show two agents used in succession, the first to find a flight and the second 

In [ ]:
from typing import Literal, Union

from pydantic import BaseModel, Field
from rich.prompt import Prompt

from pydantic_ai import Agent, RunContext
from pydantic_ai.messages import ModelMessage
from pydantic_ai.usage import Usage, UsageLimits

In [35]:
class FlightDetails(BaseModel):
    flight_number: str

class Failed(BaseModel):
    """Unable to find a satisfactory choice."""

flight_search_agent = Agent[None, Union[FlightDetails, Failed]](
    qwen_model,
    result_type=Union[FlightDetails, Failed], # type: ignore
    system_prompt=(
        'Used the "flight_search" tool to find a flight '
        'from the given origin to the given destination.'
    )
)

@flight_search_agent.tool
async def flight_search(
    ctx: RunContext[None], origin: str, destination: str
) -> Union[FlightDetails, None]:
    # in reality, this would call a flight search API or
    # use a browser to scrap a flight search website
    return FlightDetails(flight_number='AK456')

usage_limits = UsageLimits(request_limit=15)

async def find_flight(usage: Usage) -> Union[FlightDetails, None]:
    message_history: Union[list[ModelMessage], None] = None
    for _ in range(3):
        prompt = Prompt.ask(
            'Where would you like to fly from and to?'
        )
        result = await flight_search_agent.run(
            prompt,
            message_history=message_history,
            usage=usage,
            usage_limits=usage_limits            
        )
        if isinstance(result.data, FlightDetails):
            return result.data
        else:
            message_history = result.all_messages(
                result_tool_return_content='Please try again.'
            )

class SeatPreference(BaseModel):
    row: int = Field(ge=1, le=30)
    seat: Literal['A', 'B', 'C', 'D', 'E', 'F']

# This agent is responsible for extracting the user's seat selection
seat_preference_agent = Agent[None, Union[SeatPreference, Failed]](
    qwen_model,
    result_type=Union[SeatPreference, Failed], # type: ignore
    system_prompt=(
        "Extract the user's seat preference. "
        'Seats A and F are window seats. '
        'Row 1 is the front row and has extra leg room. '
        'Rows 14, and 20 also have extra leg room. '
    )
)

async def find_seat(usage: Usage) -> SeatPreference:
    message_history: Union[list[ModelMessage], None] = None
    while True:
        answer = Prompt.ask('What seat would you like?')

        result = await seat_preference_agent.run(
            answer,
            message_history=message_history,
            usage=usage,
            usage_limits=usage_limits
        )
        if isinstance(result.data, SeatPreference):
            return result.data
        else:
            print('Could not understand seat preference. Please try again.')
            message_history = result.all_messages()

usage: Usage = Usage()

opt_flight_details = await find_flight(usage)
if opt_flight_details is not None:
    print(f'Flight found: {opt_flight_details.flight_number}')
    #> Flight found: AK456
    seat_preference = await find_seat(usage)
    print(f'Seat preference: {seat_preference}')
    #> Seat preference: row=1 seat='A'

Where would you like to fly from and to?:

Flight found: AK456


What seat would you like?:

Seat preference: row=1 seat='B'
